In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import nbappinator as nbapp

matplotlib.use("agg")

PAGES = ["Chart"]

In [ ]:
def add_chart(component, action, args, app: nbapp.UiModel, caller: str):
    with app.messages:
        target_page = app.get_page(0)
        target_page.clear()

        app.update_status(name=caller, message="Drawing Chart", running=True)
        source = target_page.app.get_valuestr("source")

        df = pd.read_csv(source)

        # Create a scatter plot matrix using matplotlib
        fig, axes = plt.subplots(2, 2, figsize=(10, 10))

        colors = {"setosa": "red", "versicolor": "green", "virginica": "blue"}

        for species in df["species"].unique():
            subset = df[df["species"] == species]
            axes[0, 0].scatter(
                subset["sepal_length"], subset["sepal_width"], c=colors.get(species, "gray"), label=species, alpha=0.6
            )
            axes[0, 1].scatter(
                subset["petal_length"], subset["petal_width"], c=colors.get(species, "gray"), label=species, alpha=0.6
            )
            axes[1, 0].scatter(
                subset["sepal_length"], subset["petal_length"], c=colors.get(species, "gray"), label=species, alpha=0.6
            )
            axes[1, 1].scatter(
                subset["sepal_width"], subset["petal_width"], c=colors.get(species, "gray"), label=species, alpha=0.6
            )

        axes[0, 0].set_xlabel("Sepal Length")
        axes[0, 0].set_ylabel("Sepal Width")
        axes[0, 1].set_xlabel("Petal Length")
        axes[0, 1].set_ylabel("Petal Width")
        axes[1, 0].set_xlabel("Sepal Length")
        axes[1, 0].set_ylabel("Petal Length")
        axes[1, 1].set_xlabel("Sepal Width")
        axes[1, 1].set_ylabel("Petal Width")

        axes[0, 0].legend()
        plt.tight_layout()

        target_page.add_plt(plt=fig)

        target_page.app.update_status(name=caller, message="Done Drawing", running=False)

In [ ]:
# Creates a one "tab" app, with a config section on top, and a chart below.
# Clicking "Draw Chart" will redraw the chart with the selected data source

sources = ["https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"]

myapp = nbapp.TabbedUiModel(pages=PAGES, log_footer="Messages", headers=["Config"])

config_page = myapp.get_page("Config")

config_page.add_select(name="source", label="Source", options=sources, value=sources[0])
config_page.add_button(name="b_chart", label="Draw Chart", action=add_chart, status=True)

add_chart(None, None, None, app=myapp, caller="b_chart")
myapp.display()